In [1]:
import os
import json
from tqdm.notebook import tqdm
import cv2
import matplotlib.pyplot as plt
import uuid
import subprocess
from multiprocessing import Pool
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import shutil

In [ ]:
inputFolder = "./slices"

In [ ]:
def trim(json, size=None, augment=3, likelihood=0.98):
    
    save="./prepare/{0}.jpg"
    
    try:
        if json["likelihood"]<likelihood:
            return 
        img = cv2.imread(json["file"])
        #img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        x = json["face"]["x"]
        y = json["face"]["y"]
        width = json["face"]["width"]
        height = json["face"]["height"]
        img = img[y:y+height, x:x+width,:]
        if size:
            img = cv2.resize(img,size)

        if width>=80 and height>=80:
            cv2.imwrite(save.format(str(uuid.uuid4())), img)
            if augment:
                datagen = ImageDataGenerator(
                    rotation_range=45,
                    width_shift_range=0.1,
                    height_shift_range=0.1,
                    shear_range=0,
                    zoom_range=[0.8,1.0],
                    horizontal_flip=True,
                    vertical_flip=False,
                    fill_mode="reflect")
                img = img.reshape((1,)+img.shape)
                for i,d in enumerate(datagen.flow(img, batch_size=1)):
                    cv2.imwrite(save.format(str(uuid.uuid4())), d[0].astype(np.uint8))
                    if i==augment:
                        break
        return img
    except Exception as e:
        print(e)
        

def create_dataset(filename):
    if (filename[-4::] in (".jpg",".png"))==False :
        return
    uid = str(uuid.uuid4())
    tmpFolder="tmp-" + uid
    os.makedirs(tmpFolder)
    shutil.copy(os.path.join(inputFolder, filename), os.path.join(tmpFolder, filename))
    cmd = "ruby /animeface-2009/animeface-ruby/proc_folder.rb {0} {0}/{1}.json".format(tmpFolder, uid)
    subprocess.run(cmd.split(" "))
    with open("{0}/{1}.json".format(tmpFolder, uid), "r") as fp:
        for line in fp:
            data=json.loads(line)
            trim(data, augment=None)
    
    shutil.rmtree(tmpFolder)

In [ ]:
with Pool() as p:
    imap = p.imap(create_dataset, os.listdir(inputFolder))
    list(tqdm(imap, total=len(os.listdir(inputFolder))))